In [1]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [2]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

import dotenv

dotenv.load_dotenv()

True

In [4]:
from unstructured.partition.auto import partition
import os

pdfs_directory = "selected_files"

def preprocess_pdfs(directory):
  elements = []
  for root, _, files in os.walk(directory):
    for file in files:
        if file.endswith(".pdf"):
            elems = partition(filename=os.path.join(root,file))
            elements.extend(elems)
  return elements

final_elements = preprocess_pdfs(pdfs_directory)

print(f"Extracted {len(final_elements)} elements from the pdf")

/Users/binbasri0/Desktop/UnderDevelopment/chat-with-frro/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracted 6950 elements from the pdf


In [5]:
import re
from unstructured.cleaners.core import (
    clean, remove_punctuation, clean_non_ascii_chars,
    clean_ordered_bullets, group_broken_paragraphs
)

para_split_re = re.compile(r"(\s*\n\s*){3}")

def clean_text(x):
  # Handle the case when x.text is empty or whitespace
  if x.text.strip():  # Check if text is not empty after stripping whitespace
    x.text = clean_ordered_bullets(x.text)
    x.text = group_broken_paragraphs(x.text,paragraph_split=para_split_re)
    x.text = clean(x.text, extra_whitespace=True, bullets=True,
                      dashes=True, trailing_punctuation=True, lowercase=True)
    x.text = remove_punctuation(x.text)
    x.text = clean_non_ascii_chars(x.text)

for element in final_elements:
  clean_text(element)

In [6]:
from collections import Counter

display(Counter(type(element) for element in final_elements))

Counter({unstructured.documents.elements.NarrativeText: 2045,
         unstructured.documents.elements.Text: 2019,
         unstructured.documents.elements.Title: 1877,
         unstructured.documents.elements.ListItem: 788,
         unstructured.documents.elements.Footer: 182,
         unstructured.documents.elements.Header: 39})

In [7]:
from unstructured.chunking.title import chunk_by_title

chunks = chunk_by_title(final_elements)
print(f"Chunks created: {len(chunks)}")

Chunks created: 1603


In [8]:
print(chunks[0].metadata.to_dict())

{'file_directory': 'selected_files', 'filename': 'ForeignVisa_030214[1].pdf', 'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-08-18T04:14:38', 'page_number': 1, 'orig_elements': 'eJzdlM1unDAQx1/F4pyktjFfeYBKlaqqUtNeVitkmAEcgY3AJNmmffeOgV1FbXrMJTf7NzN4vvgfniPscUDrSwPRLYs0z1MFWVLUkCeIaRFzwQGkTCvFizSPrlg0oNegvSb/56h2bgJjtcd5vff65BZfdmjazhNRWZpRzI4fDfiOaJzEiujojPUh7nAQMuM3/Iqlmbjhxyt2AULGciOySLLXyRZEJJpPs8chVPLVPGH/bdQ1Rr/J0JgeSzAT1t5Np+AwU+W1RyiDaY52H6sHDNaPbqIS7A8z65LHXAp1EMebEZqzoz+Nq6Mex97U2htnP+zmXtt20e3akUOEto2OK519OTgwjcG115JLdc3za5HfcXUr1G28tnekyNIuQ4UTeYmQvMcnv0YkXErBueQiZg37xVr3gJMN82OuYcaC0Www1sx+OgXSuQGZbhptppk1W0lIRzAPZqaMmYW6ZqZn1WJ6GmPL7jUZbNuxyWlgFh8ZYN8Z5jtkMiezXTR9mzJQIdlzF75bagG2bjI/Ee5CupT338vF61gnqWpEqiqZYBGLDCCuIIkR0jTVb7ZcudpXSeXb4lxAyndwXq1/wBry/lfr5TA/0/58CrW+MsNKKCxEXWsldJ5nPMuanMcAPK1AqATebIYy2wYSp/sMzyARxUUN1H/IFvT+pzgv1T3lTEpACrDU4c2ZTdjT6/Rve0fnNojDmk0QiF0TmF2J7mdWu2H33eSE/AYEKqBnpBn6ctHeowVNyhPwy/X5oqfwwANuOnD8AwJJ9lw='}


In [9]:
!pip install -U langchain langchain-openai langchain_community

  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached tiktoken-0.7.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.20.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached jsonpointer-3

In [11]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

documents = []
for element in chunks:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

db = FAISS.from_documents(documents, OpenAIEmbeddings())
db.save_local("db")

In [14]:
# convert languages field to string
for document in documents:
    document.metadata["languages"] = " ".join(document.metadata["languages"])

In [19]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma(
    collection_name="frro",
    embedding_function=OpenAIEmbeddings(),
    persist_directory="frro-db"
)

chroma_db = vector_store.from_documents(documents, embedding=OpenAIEmbeddings())

In [20]:
chroma_db.persist()

/Users/binbasri0/Desktop/UnderDevelopment/chat-with-frro/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [17]:
chroma_retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = chroma_retriever.invoke("What are the approaches to Task Decomposition?")
print(retrieved_docs[0].page_content)

ii

in case of green field projects 10 of the total skilled manpower deployed per million tonne capacity or 300 persons whichever is lower for brownfield projects 5 of the total skilled manpower deployed per million tonne capacity or 150 persons whichever is lower

iii

the total manpower requirements of the project should be determined at the stage of conceptualization of the project by the sponsoring authority with a flow chart


In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
print(retrieved_docs[0].page_content)

ii

in case of green field projects 10 of the total skilled manpower deployed per million tonne capacity or 300 persons whichever is lower for brownfield projects 5 of the total skilled manpower deployed per million tonne capacity or 150 persons whichever is lower

iii

the total manpower requirements of the project should be determined at the stage of conceptualization of the project by the sponsoring authority with a flow chart


In [15]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
rag_chain.invoke("what is a Student Visa")

'A student visa is granted to a foreigner solely for pursuing full-time structured courses at recognized educational institutions. The visa may allow for changing courses a maximum of three times during its validity. Foreign nationals must obtain a fresh student visa if they need to change courses further or do not have a firm letter of admission.'

In [2]:
from langchain import hub
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
import os
os.sys.path
# loading with langchain
directory = "selected_files"
docs = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        print(os.path.join(directory, filename))
        loader = UnstructuredPDFLoader(os.path.join(directory, filename))
        docs.append(loader.load())
        
print(len(docs))


selected_files/AnnexIIIDetailsofVisas20082020[1].pdf


ModuleNotFoundError: No module named 'cv2.typing'; 'cv2' is not a package

In [10]:
# load the documents, chunck, split and embed them into a vector store using Unstructured

  0%|          | 0/2 [00:00<?, ?it/s]Error loading file /workspaces/chat-with-frro/frro-rag-notebooks/selected_files/AnnexIIIDetailsofVisas20082020[1].pdf


ModuleNotFoundError: No module named 'cv2.typing'; 'cv2' is not a package

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
# text-embedding-3-small	

In [ ]:
# Load, chunk and index the contents of the blog.


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
# cleanup
vectorstore.delete_collection()